## CBoW Modelini Eğitme

Bu not defteri, [AI for Beginners Curriculum](http://aka.ms/ai-beginners) programının bir parçasıdır.

Bu örnekte, kendi Word2Vec gömme alanımızı elde etmek için bir CBoW dil modeli eğitmeyi inceleyeceğiz. Metin kaynağı olarak AG News veri setini kullanacağız.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Öncelikle veri setimizi yükleyelim ve tokenizer ile kelime dağarcığını tanımlayalım. Hesaplamaları biraz sınırlamak için `vocab_size` değerini 5000 olarak ayarlayacağız.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW Modeli

CBoW, bir kelimeyi $2N$ komşu kelimeye dayanarak tahmin etmeyi öğrenir. Örneğin, $N=1$ olduğunda, *I like to train networks* cümlesinden şu çiftleri elde ederiz: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Burada, ilk kelime giriş olarak kullanılan komşu kelime, ikinci kelime ise tahmin etmeye çalıştığımız kelimedir.

Bir sonraki kelimeyi tahmin etmek için bir ağ oluşturmak istiyorsak, komşu kelimeyi giriş olarak sağlamamız ve kelime numarasını çıktı olarak almamız gerekir. CBoW ağının mimarisi şu şekildedir:

* Giriş kelimesi, gömme (embedding) katmanından geçirilir. Bu gömme katmanı, Word2Vec gömmemiz olacaktır, bu nedenle bunu ayrı bir `embedder` değişkeni olarak tanımlayacağız. Bu örnekte gömme boyutunu 30 olarak kullanacağız, ancak daha yüksek boyutlarla denemeler yapmak isteyebilirsiniz (gerçek Word2Vec 300 boyutludur).
* Gömme vektörü daha sonra çıktı kelimeyi tahmin edecek bir doğrusal katmana aktarılır. Bu nedenle, bu katman `vocab_size` kadar nörona sahiptir.

Çıktı için, eğer kayıp fonksiyonu olarak `CrossEntropyLoss` kullanırsak, beklenen sonuçları tek-sıcak (one-hot) kodlama olmadan, sadece kelime numaraları olarak sağlamamız gerekecektir.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Eğitim Verilerini Hazırlama

Şimdi metinden CBoW kelime çiftlerini hesaplayacak ana fonksiyonu programlayalım. Bu fonksiyon, pencere boyutunu belirlememize olanak tanıyacak ve bir çift seti - giriş ve çıkış kelimesi - döndürecek. Bu fonksiyonun hem kelimeler hem de vektörler/tensörler üzerinde kullanılabileceğini unutmayın - bu da metni `to_cbow` fonksiyonuna geçirmeden önce kodlamamıza olanak tanıyacak.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Haydi eğitim veri setini hazırlayalım. Tüm haberleri gözden geçireceğiz, kelime çiftlerinin listesini almak için `to_cbow` çağıracağız ve bu çiftleri `X` ve `Y`'ye ekleyeceğiz. Zaman kazanmak adına, yalnızca ilk 10k haber öğesini dikkate alacağız - beklemek için daha fazla zamanınız varsa ve daha iyi gömme vektörleri elde etmek istiyorsanız, bu sınırlamayı kolayca kaldırabilirsiniz :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Verileri tek bir veri kümesine dönüştüreceğiz ve veri yükleyici oluşturacağız:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Verileri tek bir veri kümesine dönüştüreceğiz ve bir veri yükleyici oluşturacağız:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Şimdi asıl eğitimi yapalım. Oldukça yüksek bir öğrenme oranına sahip `SGD` optimizasyonunu kullanacağız. Ayrıca `Adam` gibi diğer optimizasyonlarla da denemeler yapabilirsiniz. Başlangıç olarak 10 epoch boyunca eğitim yapacağız - ve daha düşük bir kayıp elde etmek isterseniz bu hücreyi tekrar çalıştırabilirsiniz.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec'i Denemek

Word2Vec'i kullanmak için, kelime dağarcığımızdaki tüm kelimelere karşılık gelen vektörleri çıkaralım:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Hadi bakalım, örneğin, **Paris** kelimesinin bir vektöre nasıl kodlandığını görelim:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Word2Vec'i eş anlamlıları aramak için kullanmak ilginçtir. Aşağıdaki fonksiyon, verilen bir girdiye en yakın `n` kelimeyi döndürecektir. Bunları bulmak için $|w_i - v|$ normunu hesaplarız, burada $v$ girdi kelimemize karşılık gelen vektör, ve $w_i$ ise sözlükteki $i$-inci kelimenin kodlamasıdır. Daha sonra diziyi sıralarız ve `argsort` kullanarak ilgili indeksleri döndürürüz, ardından sözlükteki en yakın kelimelerin pozisyonlarını kodlayan listenin ilk `n` elemanını alırız.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Özet

CBoW gibi akıllı teknikler kullanarak Word2Vec modelini eğitebiliriz. Ayrıca, merkezi bir kelime verildiğinde komşu kelimeyi tahmin etmek için eğitilen skip-gram modelini denemeyi ve ne kadar iyi performans gösterdiğini görmeyi deneyebilirsiniz.



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Orijinal belgenin kendi dilindeki hali yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan herhangi bir yanlış anlama veya yanlış yorumlama durumunda sorumluluk kabul edilmez.
